In [3]:
%matplotlib inline
import sys
import os
sys.path.insert(0,os.path.abspath(os.path.expanduser('..')))
from config_calc import *
import grid_tools

import cesm_orcas_sci as cesm
import cam
import gv

import colorbrewer

import statsmodels.api as sm
from scipy import stats

import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs

/Users/mclong/Dropbox/miniconda2/envs/py2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
ds = xr.open_dataset('../data/f09_f09.nc').isel(time=0)
ds

<xarray.Dataset>
Dimensions:    (lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 -86.23 -85.29 -84.35 ...
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 6.25 7.5 8.75 10.0 11.25 ...
    time       datetime64[ns] 2007-11-01
Dimensions without coordinates: nbnd
Data variables:
    LANDFRAC   (lat, lon) float32 ...
    OCNFRAC    (lat, lon) float32 ...
    time_bnds  (nbnd) float64 ...
Attributes:
    Conventions:      CF-1.0
    source:           CAM
    case:             bgeos5.e12.B20TRC5CN.f09_g16.frcst-ini.001
    title:            UNSET
    logname:          mclong
    host:             ys0419
    Version:          $Name$
    revision_Id:      $Id$
    initial_file:     bsd-merged.cam.r.2005-01-01-00000.nc
    topography_file:  /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/USGS-gtop...
    history:          Fri Dec  4 10:21:57 2015: ncks -O -v OCNFRAC,LANDFRAC,l...
    NCO:              4.4.2

In [ ]:
omsk = ds.LANDFRAC.where(ds.LANDFRAC. < 0.8 )

In [10]:

M = xr.full_like(ds.OCNFRAC,fill_value=0.)

my_regions = OrderedDict([
    ('1' , M.where((basin.Arctic==1.))),
    ('2' , M.where((basin.lat>49.) & (basin.Atlantic == 1.))),
    ('3' , M.where((35.<basin.lat) & (basin.lat<=49.) & (basin.Atlantic == 1.))),
    ('4' , M.where((20.<basin.lat) & (basin.lat<=35.) & (basin.Atlantic == 1.))),
    ('5' , M.where((0.<basin.lat) & (basin.lat<=20.) & (basin.Atlantic == 1.))),
    ('6' , M.where((-20.<basin.lat) & (basin.lat<=0.) & (basin.Atlantic == 1.))),
    ('7' , M.where((-35.<basin.lat) & (basin.lat<=-20.) & (basin.Atlantic == 1.))),
    ('8' , M.where((-44.<basin.lat) & (basin.lat<=-35.) & (basin.Atlantic == 1.))),
    ('9-25-30' , M.where((-58.<basin.lat) & (basin.lat<=-44.))),
    ('10' , M.where((-90.<basin.lat) & (basin.lat<=-58.))),
    ('11' , M.where((50.<basin.lat) & (basin.lat<=70.) & ~((basin.lon<0.) & (basin.lon>-170.)) & (basin.Pacific == 1.))),
    ('12' , M.where((40.<basin.lat) & (basin.lat<=70.) & ((basin.lon<0.) & (basin.lon>-170.)) & (basin.Pacific == 1.))),
    ('13-14-15' , M.where(((20.<basin.lat) & (basin.lat<=50.) & ~((basin.lon<0.) & (basin.lon>-170.)) & (basin.Pacific == 1.)) |
                        ((20.<basin.lat) & (basin.lat<=40.) & ((basin.lon<0.) & (basin.lon>-170.)) & (basin.Pacific == 1.)))),
    ('16' , M.where((0.<basin.lat) & (basin.lat<=20.) & ~((basin.lon<0.) & (basin.lon>-160.)) & (basin.Pacific == 1.))),
    ('17' , M.where((0.<basin.lat) & (basin.lat<=20.) & ((basin.lon<0.) & (basin.lon>-160.)) & (basin.Pacific == 1.))),
    ('18' , M.where((-20.<basin.lat) & (basin.lat<=0.) & ~((basin.lon<0.) & (basin.lon>-160.)) & (basin.Pacific == 1.))),
    ('19' , M.where((-20.<basin.lat) & (basin.lat<=0.) & ((basin.lon<0.) & (basin.lon>-160.)) & (basin.Pacific == 1.))),
    ('20-22' , M.where(((-30.<basin.lat) & (basin.lat<=-20.) & ~((basin.lon<0.) & (basin.lon>-130.)) & (basin.Pacific == 1.)) |
                      ((-44.<basin.lat) & (basin.lat<=-30.) & ~((basin.lon<0.) & (basin.lon>-112.)) & (basin.Pacific == 1.)))),
    ('21-23-24' , M.where(((-30.<basin.lat) & (basin.lat<=-20.) & ((basin.lon<0.) & (basin.lon>-130.)) & (basin.Pacific == 1.)) |
                         ((-44.<basin.lat) & (basin.lat<=-30.) & ((basin.lon<0.) & (basin.lon>-112.)) & (basin.Pacific == 1.)))),
    ('26-27' , M.where((-20.<basin.lat) & (basin.lat<=50.) & (basin.Indian == 1.))),
    ('28-29' , M.where((-44.<basin.lat) & (basin.lat<=-20.) & (basin.Indian == 1.)))])

nregion = len(my_regions)
print('Number of regions: %d'%nregion)
rmask = xr.DataArray(np.zeros((nregion,len(ds.lat),len(ds.lon))),dims=('region','lat','lon'),
                     coords=ds.coords)

for i,mask_logic in enumerate(my_regions.values()):
    rmask.values[i,:,:] = mask_logic.fillna(0.)
rmask

<xarray.DataArray (region: 5, lat: 192, lon: 288)>
array([[[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       ...,

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]]])
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 -86.23 -85.29 -84.35 ...
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 6.25 7.5 8.75 10.0 11.25 ...
    time     datetime64[ns] 2007-11-01
Dimensions without coordinates: region